In [1]:
import os
import pickle as pickle
import tensorflow as tf
import numpy as np
from Nets  import GetNeuralNetworkModel
from StatsReadWrite import WeightQuantization, ActivationStats, CheckAccuracyAndLoss, QuantizationEffect, GetReadAndWrites
from Training import GetDatasets, GetPilotNetDataset
import pandas as pd
from datetime import datetime
from Simulation import buffer_simulation, save_obj, load_obj
from funciones import VeryBadWords,FlipPatchBetter


In [2]:
error_mask = load_obj('MoRS/Modelo3_mas_fallos_col_8_experimentos/mask/error_mask_0')
locs = load_obj('MoRS/Modelo3_mas_fallos_col_8_experimentos/mask/locs_0')

In [3]:
word_size  = 16
afrac_size = 11
aint_size  = 4
wfrac_size = 11
wint_size  = 4
trainBatchSize = testBatchSize = 1
_, _, test_dataset = GetDatasets('colorectal_histology', (80, 5, 15), (227, 227), 8, trainBatchSize, testBatchSize)


cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'AlexNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')


C:\Users\usuario\anaconda3\envs\env_first\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


In [4]:
activation_aging = [True]*11
AlexNet = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                                aging_active=activation_aging, word_size=word_size, frac_size=afrac_size,
                               batch_size = testBatchSize)
AlexNet.load_weights(wgt_dir).expect_partial()
WeightQuantization(model=AlexNet, frac_bits=wfrac_size, int_bits=wint_size)
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
AlexNet.compile(optimizer=optimizer, loss=loss, metrics='accuracy')



#loss,acc =AlexNet.evaluate(test_dataset)

x KerasTensor(type_spec=TensorSpec(shape=(None, 227, 227, 3), dtype=tf.float32, name=None), name='lambda_1/TensorScatterUpdate:0', description="created by layer 'lambda_1'")


In [15]:
input_shape = []
layer_name = []
for index, layer in enumerate(AlexNet.layers):
    layer_name.append(layer.name)
    input_shape.append(layer.input_shape[1:])
    # print(index,layer.name)
    # print(index,layer.input_shape[1:])

df_layer_name = pd.DataFrame(layer_name)
df_layer_size = pd.DataFrame(input_shape)
print(df_layer_name,df_layer_size)
result = pd.concat([df_layer_name,df_layer_size], axis=1, join='outer')
result.columns =['layer_name', 'h','w','ch']
result.to_excel('AlexNet_shape_layers.xlsx',  index=False)
print(result)

                          0
0                   input_1
1                    lambda
2                  lambda_1
3                     Conv1
4                  lambda_2
5                     re_lu
6       batch_normalization
7                  lambda_3
8                  lambda_4
9             max_pooling2d
10                 lambda_5
11                    Conv2
12                 lambda_6
13                  re_lu_1
14    batch_normalization_1
15                 lambda_7
16                 lambda_8
17          max_pooling2d_1
18                 lambda_9
19                    Conv3
20                lambda_10
21                  re_lu_2
22    batch_normalization_2
23                lambda_11
24                lambda_12
25                    Conv4
26                lambda_13
27                  re_lu_3
28    batch_normalization_3
29                lambda_14
30                lambda_15
31                    Conv5
32                lambda_16
33                  re_lu_4
34    batch_normaliz

In [5]:
num_address  =1048576
Indices      = [0,3,9,11,17,19,25,31,37,40,45,50] #Capas con la informacion de procesamiento
#locs_VBW = [101102, 124520, 302065, 331634, 340492, 340493, 360087, 360088, 377722, 377723, 419841, 458633, 458634, 465007, 465008, 465034, 465197, 465389, 544769, 544770, 545758, 590341, 590402, 590403, 590404, 590405, 590406, 590407, 590408, 590409, 590424, 590610, 590706, 590707, 590790, 590804, 611329, 807694, 596228, 371814, 153245, 431863, 870432, 431606, 854134, 464132, 75599, 984684, 957428, 144540, 155294, 449309, 343438, 224550, 515399, 278216, 103082, 153193, 910945, 685437, 504077, 450176, 49594, 699634, 734341, 635221, 639518, 276509, 385631, 40833, 887620, 259787, 105166, 487082, 421367, 829519, 1002831, 827753, 750402, 342656, 641683, 132117, 846177, 670210, 507666, 183528, 108520, 335181, 645235, 306439, 986119, 910692, 870824, 260319, 435744, 768399, 94637, 525999, 1026694, 26510, 816438, 225078, 190841, 422703, 463124, 467197, 1030806, 379426, 871962, 746460, 360883, 971049, 559437, 989409, 145877, 845559, 1018805, 283649, 79627, 912268, 1042255, 676817, 309244, 682316, 493406, 151515, 58733, 403778, 402881, 793085, 416518, 4606, 305748, 143466, 16917, 28154, 504505, 91708, 1013618, 350501, 367555, 993020, 563837, 128, 77845, 697509, 448560, 25033]

samples      = 1 #Numero de imagenes
# Sin Power Gating:
Data         = GetReadAndWrites(AlexNet,Indices,num_address,samples,CNN_gating=False,network_name='AlexNet')
stats        = {'Lecturas': Data['Reads'],'Escrituras': Data['Writes']}
Baseline_Acceses   = pd.DataFrame(stats).reset_index(drop=False)
df_writes_Read =  pd.concat([Baseline_Acceses], axis=1, join='outer')
df_writes_Read .columns = ['index','Lecturas','Escrituras']

network AlexNet
[0, 3, 9, 11, 17, 19, 25, 31, 37, 40, 45, 50] [0, 3, 9, 11, 17, 19, 25, 31, 37, 40, 45, 50]
addresses_entrada 1048576
capas de escrituras
 capa, tamaño InputLayer 154587
suma de las escrituras [0 0 0 ... 0 0 0]
escrituras bloque de 16 9661.6875
buffer_divitions [0, 131072, 262144, 393216, 524288, 655360, 786432, 917504, 1048576]
bid 1
capas de leturas
 capa Conv2D
layer==4 Conv2D
shape de estas capas.......................................... (None, 227, 227, 3)
 capa, tamaño Conv2D 154587
suma de las lecturas no las calculo aquí, debo analizar 
lecturas bloque de 16 9661.6875
padding................ valid
dentro de valid
layer.strides (4, 4)
recorrido del kernel total y 55
recorrido del kernel total x 3025
Kernel_Y1 220
Kernel_Y2 231
condicional conv2D.......................................
Conv2D Conv2D
bid 0
capas de escrituras
 capa, tamaño MaxPooling2D 69984
suma de las escrituras [1 1 1 ... 1 1 1]
escrituras bloque de 16 4374.0
buffer_divitions [0, 131072, 262144, 

PermissionError: [Errno 13] Permission denied: 'read_write_blocks_testAlexNet.xlsx'

In [12]:
Buffer  = load_obj('Data/Stats/AlexNet/cycles/buffer')
ciclos  = load_obj('Data/Stats/AlexNet/cycles/cycles')
config  = load_obj('Data/Stats/AlexNet/cycles/config')

In [10]:
ciclos

3192016

In [11]:
Buffer

{'Number of Addresses': 524288,
 'Data': array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
 'HighCyclesCount': array([3541616, 1222405,  637804, ...,       0,       0,       0],
       dtype=uint32),
 'OffCyclesCount': array([0, 0, 0, ..., 0, 0, 0], dtype=uint32),
 'LowCyclesCount': array([4294617696,    1969611,    2554212, ...,    3192016,    3192016,
           3192016], dtype=uint32),
 'Flips': array([2, 2, 2, ..., 0, 0, 0], dtype=uint32),
 'offset': 0}

In [13]:
config

{'word size': 16,
 'frac size': 11,
 'invert bits': 0,
 'bitshift': 0,
 'CNN-Gated': False,
 'write mode': 'default',
 'Number of switchable sections': 8}

In [6]:
samples = 1
LI = [0,3,9,11,17,19,25,31,37,40,45,50]
AI = [2,8,10,16,18,24,30,36,38,44,49,53]
Buffer,ciclos =  buffer_simulation(AlexNet, test_dataset, integer_bits = 4, fractional_bits = 11, samples = samples, start_from = 0,
                                  bit_invertion = False, bit_shifting = False, CNN_gating = False,
                                  buffer_size = 1048576, write_mode ='default', save_results = True, network_type = 'AlexNet',
                                  results_dir = 'Data/Stats/AlexNet/mask_x/',
                                  layer_indexes = LI , activation_indixes = AI)

print(str()+' operación ciclos completada: ', datetime.now().strftime("%H:%M:%S"))

buffer sections:  [0, 65536, 131072, 196608, 262144, 327680, 393216, 458752, 524288]
Simulation Started, time: 12:41:40 cycles:  0 offset:  0
--now processing CPULayer input_1
----layer size:  154587
------Current Address:  0
------Wrap:  False


CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:

In [13]:
word_size  = 16
afrac_size = 9
aint_size  = 6
wfrac_size = 15
wint_size  = 0

trainBatchSize = testBatchSize = 1
_, _, test_dataset = GetDatasets('colorectal_histology', (80, 5, 15), (224, 224), 8, trainBatchSize, testBatchSize)


cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'SqueezeNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')


C:\Users\usuario\anaconda3\envs\env_first\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


In [14]:
activation_aging = [True]*22


#Acá la creamos, notese que como no se introduciran fallos en activaciones no es necesario pasar locs ni masks
SqueezeNet = GetNeuralNetworkModel('SqueezeNet', (224,224,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                                 aging_active=activation_aging, word_size=word_size, frac_size=afrac_size,
                                 batch_size = testBatchSize)

loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
SqueezeNet.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
SqueezeNet.load_weights(wgt_dir).expect_partial()
WeightQuantization(model=SqueezeNet, frac_bits=wfrac_size, int_bits=wint_size)

In [15]:
num_address  =1048576
samples      = 1
Indices = [0,3,7, 9,(13,14),20,(24,25),31,(35,36),42,44,(48,49),55,(59,60),66,(70,71),77,(81,82),88,90,(94,95),101,104]
Data    = GetReadAndWrites(SqueezeNet,Indices,num_address,samples,CNN_gating=False,network_name='SqueezeNet')
stats   = {'Lecturas': Data['Reads'],'Escrituras': Data['Writes']}
Baseline_Acceses   = pd.DataFrame(stats).reset_index(drop=False)
df_writes_Read =  pd.concat([Baseline_Acceses], axis=1, join='outer')
df_writes_Read.columns = ['index','Lecturas','Escrituras']

network SqueezeNet
[0, 3, 7, 9, (13, 14), 20, (24, 25), 31, (35, 36), 42, 44, (48, 49), 55, (59, 60), 66, (70, 71), 77, (81, 82), 88, 90, (94, 95), 101, 104] [0, 3, 7, 9, (13, 14), 20, (24, 25), 31, (35, 36), 42, 44, (48, 49), 55, (59, 60), 66, (70, 71), 77, (81, 82), 88, 90, (94, 95), 101, 104]
addresses_entrada 1048576
capas de escrituras
 capa, tamaño InputLayer 150528
suma de las escrituras [0 0 0 ... 0 0 0]
escrituras bloque de 16 9408.0
buffer_divitions [0, 131072, 262144, 393216, 524288, 655360, 786432, 917504, 1048576]
bid 1
capas de leturas
 capa Conv2D
layer==4 Conv2D
shape de estas capas (None, 224, 224, 3)
 capa, tamaño Conv2D 150528
suma de las lecturas no las calculo aquí, debo analizar 
lecturas bloque de 16 9408.0
dentro del elese por tanto padding!=valid
recorrido del kernel
recorrido del kernel
recorrido del kernel
recorrido del kernel
recorrido del kernel
recorrido del kernel
recorrido del kernel
recorrido del kernel
recorrido del kernel
recorrido del kernel
recorrid

capas de escrituras
 capa, tamaño Conv2D 401408
suma de las escrituras [6 6 6 ... 0 0 0]
escrituras bloque de 16 25088.0
buffer_divitions [0, 131072, 262144, 393216, 524288, 655360, 786432, 917504, 1048576]
capas de escrituras
 capa, tamaño Conv2D 401408
suma de las escrituras [7 7 7 ... 1 1 1]
escrituras bloque de 16 25088.0
buffer_divitions [0, 131072, 262144, 393216, 524288, 655360, 786432, 917504, 1048576]
bid 1
capas de leturas
 capa MaxPooling2D
layer.input_shape[1:]) (56, 56, 256)
actual_values [5 5 5 ... 0 0 0]
shape de estas capas (None, 56, 56, 256)
 capa, tamaño MaxPooling2D 802816
suma de las lecturas [5 5 5 ... 0 0 0]
lecturas bloque de 16 50176.0
bid 0
capas de escrituras
 capa, tamaño Conv2D 25088
suma de las escrituras [8 8 8 ... 8 8 8]
escrituras bloque de 16 1568.0
buffer_divitions [0, 131072, 262144, 393216, 524288, 655360, 786432, 917504, 1048576]
capas de leturas
 capa Conv2D
layer==4 Conv2D
shape de estas capas (None, 28, 28, 32)
 capa, tamaño Conv2D 25088
suma de

capas de escrituras
 capa, tamaño Conv2D 50176
suma de las escrituras [13 13 13 ... 10 10 10]
escrituras bloque de 16 3136.0
buffer_divitions [0, 131072, 262144, 393216, 524288, 655360, 786432, 917504, 1048576]
capas de escrituras
 capa, tamaño Conv2D 50176
suma de las escrituras [14 14 14 ... 11 11 11]
escrituras bloque de 16 3136.0
buffer_divitions [0, 131072, 262144, 393216, 524288, 655360, 786432, 917504, 1048576]
bid 1
capas de leturas
 capa Conv2D
layer==4 Conv2D
shape de estas capas (None, 14, 14, 512)
 capa, tamaño Conv2D 100352
suma de las lecturas no las calculo aquí, debo analizar 
lecturas bloque de 16 200704.0
dentro del elese por tanto padding!=valid
recorrido del kernel
recorrido del kernel
recorrido del kernel
recorrido del kernel
recorrido del kernel
recorrido del kernel
recorrido del kernel
recorrido del kernel
recorrido del kernel
recorrido del kernel
recorrido del kernel
recorrido del kernel
recorrido del kernel
recorrido del kernel
condicional conv2D...............